## _ 01. Импорт библиотек.

In [ ]:
import numpy as np                                      
import pandas as pd                                     
import matplotlib.pyplot as plt                        
import seaborn as sns                                   
                

from pandas import DataFrame                           

from sklearn.preprocessing import LabelEncoder          
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, PowerTransformer, Normalizer    
from sklearn.model_selection import train_test_split    
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error 

from sklearn.neighbors import KNeighborsRegressor       
from sklearn.svm import SVR                             
from sklearn.linear_model import LinearRegression       
from sklearn.tree import DecisionTreeRegressor          
from sklearn.ensemble import AdaBoostRegressor          
from sklearn.ensemble import GradientBoostingRegressor  
from xgboost import XGBRegressor                        
from sklearn.ensemble import RandomForestRegressor      
from sklearn.linear_model import SGDRegressor           
from sklearn.linear_model import Lasso                  

from scipy import stats                                 
from scipy.stats import shapiro                         

from sklearn.pipeline import make_pipeline              
from sklearn.model_selection import GridSearchCV       

import tensorflow.keras as keras                        
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
# -- my -- no import -- Error
from tensorflow import keras
try:
    import scikeras
except ImportError:
    !python -m pip install scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor

from joblib import dump, load                           
from tensorflow.keras.models import load_model          

import warnings                                         
warnings.filterwarnings("default", category=FutureWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import flask                                            
from flask import Flask, request, render_template

%matplotlib inline  

In [ ]:
!pip install xgboost

In [ ]:
pip install scikeras[tensorflow]

## _ 02. Загрузка и обработка исходных датасетов.

In [ ]:
!rm -r /content/sample_data
# TODO: Сделать загрузку по ссылке, а не вручную.

In [ ]:
dataset_bp = pd.read_excel('/content/X_bp.xlsx')
dataset_bp.head()

In [ ]:
dataset_bp.shape

In [ ]:
dataset_bp.tail()

In [ ]:
# 1
dataset_bp = pd.read_excel('/content/X_bp.xlsx', index_col=0)
dataset_bp.head()

In [ ]:
dataset_bp.shape

In [ ]:
# TODO: Сделать загрузку по ссылке, а не вручную.
dataset_nup = pd.read_excel('/content/X_nup.xlsx')
dataset_nup.head()

In [ ]:
dataset_nup.shape

In [ ]:
# 2
dataset_nup.drop(['Unnamed: 0'], axis=1, inplace=True)
dataset_nup.head()

In [ ]:
dataset_nup.shape

In [ ]:
# inner join
dataset = pd.merge(dataset_bp, dataset_nup, 
                   left_index=True, 
                   right_index=True, 
                   how = "inner")
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.to_excel('/content/X_full.xlsx')

In [ ]:
df = dataset.copy()

## _ 03. Разведочный анализ данных.

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
df['Угол нашивки, град'].value_counts()

In [ ]:
df['Угол нашивки, град'] = df['Угол нашивки, град'].map({0.0:0, 90.0:1}).astype(int)

In [ ]:
df = df.rename(columns={'Угол нашивки, град': 'Угол нашивки'})
df

In [ ]:
column_names = df.columns
column_names

In [ ]:
df.describe()

In [ ]:
mean_median = pd.DataFrame({'Среднее': df.mean(axis=0),
                            'Медиана': df.median(axis=0),
                            'Разница м/у средним и медианой': df.mean(axis=0) - df.median(axis=0)})
mean_median

In [ ]:
# 1. 
df.corr(method = 'pearson')

Вывод: Коэффициент корреляции Пирсона. Cущественная зависимость между пременными отсутствует.

In [ ]:
# 2. 
df.corr(method = 'spearman')

Вывод: Корреляция Спирмена. Зависимость между переменными не наблюдается.

In [ ]:
# 3.
df.corr(method = 'kendall')

Вывод: Коэффициент корреляции Кендалла. Статистическая зависимость между пременными отсутствует.

In [ ]:
# viz 1.
df.hist(figsize = (15,15), color = 'darkmagenta')
plt.suptitle('Гистограммы распределения', fontsize=20,  y=0.95)
plt.show()

In [ ]:
# viz 2. 
n_subplots = len(df.columns)                        
n_cols = 3                                          
n_rows = (n_subplots + n_cols - 1) // n_cols        

plt.figure(figsize=(15,20))
plt.suptitle('Гистограммы и графики плотности', fontsize=20,  y=0.92)
for i, col in enumerate(df.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(df[col], kde=True, color='darkmagenta') 
    plt.ylabel("")
plt.show()

In [ ]:
# viz 3.
n_subplots = len(df.columns)                        
n_cols = 4                                          
n_rows = (n_subplots + n_cols - 1) // n_cols        

plt.figure(figsize=(20,25))
plt.suptitle('Диаграммы "ящик с усами"', fontsize=20,  y=0.92)
for i, col in enumerate(df.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.boxplot(data=df[col], color='mediumorchid') 
    plt.title(df[col].name, size=12)
plt.show()

In [ ]:
# viz 4. 
plt.figure(figsize=(20,20))
sns.set(style='white', palette='RdPu_r')
sns.pairplot(df)
plt.show()

In [ ]:
# viz 5.
plt.figure(figsize=(20,20))
sns.set(style='white', palette='RdPu_r')
sns.pairplot(df, hue='Угол нашивки')
plt.show()

In [ ]:
# viz 6. 
plt.figure(figsize=(20,20))
sns.set(style='white', palette='RdPu_r')

g = sns.PairGrid(df)
g.map_upper(plt.scatter)                   
g.map_diag(sns.histplot, kde=True)         
g.map_lower(sns.kdeplot)                   
plt.show()

In [ ]:
# viz 7. 
f, ax = plt.subplots(figsize=(12,10))
sns.heatmap(df.corr(), annot=True, ax=ax, square = True, cmap='RdPu')
plt.show()

### _ 04.1. Работа с выбросами.

TODO m1

-----------------------------------------------------------------------------------------------------------------

TODO m2

todo Itogo.

Вывод: выбросы из данных удалены после трех этапов очистки.

### _ 04.2. Нормализация и стандартизация данных.

Проверку на нормальность можно проводить несколькими способами:

* построение гистограммы признака
* построение QQ-графика
* проведение теста на нормальность

#### _ 04.2.1. Проверка на нормальность.

#### _ 04.2.2. Нормализация данных.

##### _04.2.2.1. MinMaxScaler.



##### _ 04.2.2.2. RobustScaler.

#### _ 04.2.3. Стандартизация данных.

##### _ 04.2.3.2. StandardScaler. 

##### _ 04.2.3.3. Normalizer.

Все значения теперь находятся в диапазоне от 0 до 1.

##### _ 04.2.3.4. PowerTransformer.

In [ ]:
# 1.5. Преобразование Йео-Джонсона из метода PowerTransformer()
yeo = PowerTransformer(method='yeo-johnson', standardize=True)                          
dataset_norm_yeo = yeo.fit_transform(np.array(df_clean[df_clean.columns]))              
df_norm_yeo = pd.DataFrame(data = dataset_norm_yeo, columns = df_clean.columns)         
df_norm_yeo.describe()                                                                  

In [ ]:
plt.figure(figsize=(15,12))
sns.set(context='notebook', style='whitegrid')
sns.kdeplot(data=df_norm_yeo)
plt.show()

#### _04.2.4. Проверка на нормальность после нормализации.

In [ ]:
# 1. Тест Шапиро-Уилка на нормальность
for col in df_norm_minmax.columns:
    print(df_norm_minmax[col].name, shapiro(df_norm_minmax[col]))

In [ ]:
# 2. Гистограммы расределения после нормализации
df_norm_minmax.hist(figsize = (15,15), color = 'mediumorchid')
plt.suptitle('Гистограммы после нормализации', fontsize=20,  y=0.95)
plt.show()

In [ ]:
# 3. QQ-графики после нормализации
n_subplots = len(df_norm_minmax.columns)                  
n_cols = 4                                                
n_rows = (n_subplots + n_cols - 1) // n_cols              

plt.figure(figsize=(20,25))
plt.suptitle('Распределение после нормализации', fontsize=20,  y=0.92)
for i, col in enumerate(df_norm_minmax.columns, 1):
    plt.subplot(n_rows, n_cols, i)
    stats.probplot(df_norm_minmax[col], dist='norm', plot=plt)
    plt.title(df_norm_minmax[col].name, size=12)
plt.show()

TODO  выводы

## _ 05.  Модели для прогноза модуля упругости при растяжении и прочности при растяжении.

В случае прогнозирования модуля упругости при растяжении и прочности при растяжении будет решаться задача регрессии. Задача регрессии – это задача обучения с учителем.

### _ 05.1. Прогнозирование модуля упругости.

In [ ]:
print('Размер тренировочного датасета: {}\nРазмер тестового датасета:{}'.format(X_train_elastic.shape, X_test_elastic.shape))

In [ ]:
X_train_elastic

In [ ]:
y_train_elastic

TODO  выводы 

### _05.2. Прогнозирование прочности при растяжении.

TODO  выводы

## _06. Нейронная сеть для рекомендации соотношения "матрица - наполнитель".

## _07. Разработка приложения для прогнозирования соотношения "матрица-наполнитель"

## _08. Выводы: